In [ ]:
import pyTigerGraph as tg
import dgl
import networkx as nx
import torch
from py_scripts.gat import GAT
import torch.nn as nn
import torch.nn.functional as F
import py_scripts.cfg as cfg
import gensim
import time
import numpy as np

In [ ]:
numEpochs = 1000
learningRate = 0.01
wantedTopic = "Saxophone"
unwantedTopic = "Falkland_Islands" #In future, determine automatically through the inverse of PageRank/other centrality algo

In [ ]:
conn = tg.TigerGraphConnection(ipAddress="https://wikipediagraph.i.tgcloud.us", apiToken=cfg.token, graphname="WikipediaGraph") # connection to TigerGraph database

articleToNum = {} # translation dictionary for article name to number (for dgl)
numToArticle = {} # translation dictionary for number to article name
i = 0
def createEdgeList(result): # returns tuple of number version of edge
    global i
    if result["article1"] in articleToNum:
        fromKey = articleToNum[result["article1"]]
    else:
        articleToNum[result["article1"]] = i
        numToArticle[i] = result["article1"]
        fromKey = i
        i+=1
    if result["article2"] in articleToNum:
        toKey = articleToNum[result["article2"]]
    else:
        articleToNum[result["article2"]] = i
        numToArticle[i] = result["article2"]
        toKey = i
        i+=1
    return (fromKey, toKey)
    
edges = [createEdgeList(thing) for thing in conn.runInstalledQuery("tupleArticle", {})["results"][0]["list_of_article_tuples"]] # creates list of edges


In [ ]:
wantedNum = articleToNum[wantedTopic]
unwantedNum = articleToNum[unwantedTopic]

g = nx.DiGraph()

g.add_edges_from(edges)

G = dgl.DGLGraph(g)

print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())

In [ ]:
d2v_model = gensim.models.doc2vec.Doc2Vec.load("doc2vec.model")
vectors = []
for article in articleToNum:
    try: 
        words = [] 
        with open(("plaintext_articles/"+article+".txt"), "r") as f: 
            words = f.read().lower().split() 
        vectors.append(list(d2v_model.infer_vector(words)))
    except:
        vectors.append([0 for i in range(300)])

G.ndata["feat"] = torch.tensor(vectors)

print(G.nodes[2].data['feat'])

In [ ]:
net = GAT(G, G.n)